In [1]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from collections import Counter
from functools import wraps
import logging
# !pip3 install openpyxl

In [2]:
# logger = logging.getLogger()
# fhandler = logging.FileHandler(filename='mylog.log', mode='a')
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# fhandler.setFormatter(formatter)
# logger.addHandler(fhandler)
# logger.setLevel(logging.DEBUG)
# logging.error('hello!')
# logging.debug('This is a debug message')
# logging.info('this is an info message')
# logging.warning('tbllalfhldfhd, warning.')

# Util Functions

In [3]:
def log_fn(function):
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            res = f(*args, **kwargs)
            logging.info("Testando funcao")
            print(function)
            return res
        return wrapper
    return decorator


@log_fn("download_data")
def download_data(data_sources):
    to_remove = []
    for file_name, download_link in data_sources.items():
        try:
            print(f"Downloading: {file_name}, from: {download_link}")
            df = pd.read_excel(download_link)
            df.to_csv(f"../input/{file_name}.csv")
            del(df)
        except:
            to_remove.append(file_name)
            print(f"Download error for file {file_name}, link invalido {download_link}")
    data_sources = to_remove_missing_file(data_sources, to_remove)
    return data_sources


@log_fn("to_remove_missing_file")
def to_remove_missing_file(data_sources, to_remove):
    for remove in to_remove:
        data_sources.pop(remove)
    return data_sources


@log_fn("create_architecture_table")
def create_architecture_table(df, columns_description):
    architecture = pd.DataFrame(
        columns = [
            "nome",
            "tipo",
            "descricao",
            "cobertura_temporal",
            "dicionario",
            "coluna_diretorio",
            "unidade_medida",
            "nome_original"
        ]
    )
    for index, column in enumerate(df.columns):
        new_row = {
            'nome': column.lower().replace(" ", "_"),
            'tipo': df.dtypes[column] if df.dtypes[column] != object else 'string',
            'descricao': columns_description.iloc[:, index].values[0],
            'cobertura_temporal': "1996~2012",
            'dicionario': np.nan,
            'coluna_diretorio': np.nan,
            'unidade_medida': np.nan,
            'nome_original': column
        }
        architecture = architecture.append(new_row, ignore_index=True)
    return architecture


@log_fn("rename_column_guide")
def rename_column_by_guide(df):
    df_columns = df.columns.str.lower()
    df_columns = df_columns.str.replace(" ", "_")
    return df_columns

---
# Download data to input folder

In [4]:
data_sources = {
    "chronic_g_day_to_t_pop": 'http://www.efsa.europa.eu/sites/default/files/chronicgdaytotpop.xlsx',
    "chronic_g_day_bw_to_t_pop": 'http://www.efsa.europa.eu/sites/default/files/chronicgdaybwtotpop.xlsx',
    "chronic_g_day_consumers": 'http://www.efsa.europa.eu/sites/default/files/chronicgdayconsumers.xlsx',
    "chronic_g_day_bw_consumers": 'http://www.efsa.europa.eu/sites/default/files/chronicgdaybwconsumers.xlsx',
    "acute_g_day_all_days": 'https://dwh.efsa.europa.eu/bi/asp/Main.aspx?rwtrep=008',
    "acute_g_day_bw_all_days": 'http://www.efsa.europa.eu/sites/default/files/acutegdaybwalldays.xlsx',
    "acute_g_day_cons_days": 'http://www.efsa.europa.eu/sites/default/files/acutegdayconsdays.xlsx',
    "acute_g_day_bw_cons_days": 'http://www.efsa.europa.eu/sites/default/files/acutegdaybwconsdays.xlsx',
    "ESFA_CONSUMPTION_DASHBOARD_FORMAT": 'https://data.europa.eu/euodp/repository/ec/efsa/ESFA_CONSUMPTION_DASHBOARD_FORMAT.xlsx',
}
data_sources = download_data(data_sources)

Downloading: chronic_g_day_to_t_pop, from: http://www.efsa.europa.eu/sites/default/files/chronicgdaytotpop.xlsx
Downloading: chronic_g_day_bw_to_t_pop, from: http://www.efsa.europa.eu/sites/default/files/chronicgdaybwtotpop.xlsx
Downloading: chronic_g_day_consumers, from: http://www.efsa.europa.eu/sites/default/files/chronicgdayconsumers.xlsx
Downloading: chronic_g_day_bw_consumers, from: http://www.efsa.europa.eu/sites/default/files/chronicgdaybwconsumers.xlsx
Downloading: acute_g_day_all_days, from: https://dwh.efsa.europa.eu/bi/asp/Main.aspx?rwtrep=008
Download error for file acute_g_day_all_days, link invalido https://dwh.efsa.europa.eu/bi/asp/Main.aspx?rwtrep=008
Downloading: acute_g_day_bw_all_days, from: http://www.efsa.europa.eu/sites/default/files/acutegdaybwalldays.xlsx
Downloading: acute_g_day_cons_days, from: http://www.efsa.europa.eu/sites/default/files/acutegdayconsdays.xlsx
Downloading: acute_g_day_bw_cons_days, from: http://www.efsa.europa.eu/sites/default/files/acutegd

---
# Load data

In [5]:
loaded_files = {}
columns_description = {}

In [6]:
for file_name, link in data_sources.items():
    input_file = f"../input/{file_name}.csv"
    loaded_files[file_name] = pd.read_csv(input_file, header=2).iloc[:,1:]
    columns_description[file_name] = pd.read_csv(input_file, nrows = 1).iloc[:,1:]

/home/martoso/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3012: DtypeWarning: Columns (12,14,16,18,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


---
# Create Architecture Table

In [7]:
table_architecture = {}

In [8]:
for file_name, dataframe in loaded_files.items():
    table_architecture[file_name] = create_architecture_table(dataframe, columns_description[file_name])

create_architecture_table
create_architecture_table
create_architecture_table
create_architecture_table
create_architecture_table
create_architecture_table
create_architecture_table
create_architecture_table


---
# Change columns name

In [9]:
for file_name, dataframe in loaded_files.items():
    dataframe.columns = rename_column_by_guide(dataframe)

rename_column_guide
rename_column_guide
rename_column_guide
rename_column_guide
rename_column_guide
rename_column_guide
rename_column_guide
rename_column_guide


---
# Round data to 2 decimal places

In [10]:
for file_name, dataframe in loaded_files.items():
    dataframe = dataframe.round(2)

---
# Save Correct dataframe to output

In [11]:
for file_name, dataframe in loaded_files.items():
    dataframe.to_parquet(f"../output/{file_name}.parquet")

In [12]:
for file_name, dataframe in table_architecture.items():
    dataframe.to_csv(f"../output/dictionary/{file_name}.csv", index=False)